# CS848: The art and science of empirical computer science

## Assignment: Visualization Project

**Description:** At a high-level, for this visualization project, I would like you to perform exploratory data analysis on a bibliometric dataset of your choice. From this exploration, I would like you to come up with one or more interesting observations or questions to ask. And then, I would like you to build a visualization that either "makes the points" or answers the questions that you posed.

### 1. Exploratory data analysis on a bibliometric dataset of your choice:


* The data set I chose is [DBLP](https://qlever.cs.uni-freiburg.de/dblp/jzdksf). 
* [DBLP](https://qlever.cs.uni-freiburg.de/dblp/jzdksf) allows you to run SPARQL queries to get bibliometrics about:
   * Papers
   * Their authors
   * The affiliation of the authors
   * Conferences in which those papers are published
   * etc.

### 2. Observations or questions to ask:
[CSRankings](https://csrankings.org/) provide **per institution** rankings for different CS fields.
We are interested in a similar ranking but **per country**.

More specifically, for **a given field** (for example: cloud), **What are the rankings by country?**


### How to answer this question?